In [74]:
texto=spark.createDataFrame(
    [
        (0,"Hola, quiero aprender algo de Spark"),
        (1,"Me gustaría aprender algo de Java"),
        (2,"suma,Resta,multiplicación")
    ],
    ["id","frase"]
)

In [75]:
texto.show()

+---+--------------------+
| id|               frase|
+---+--------------------+
|  0|Hola, quiero apre...|
|  1|Me gustaría apren...|
|  2|suma,Resta,multip...|
+---+--------------------+



In [76]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer

In [77]:
tokenizer=Tokenizer(inputCol="frase",outputCol="palabras")

In [78]:
datosT=tokenizer.transform(texto)

In [79]:
datosT.show(truncate=False,vertical=True)

-RECORD 0----------------------------------------------
 id       | 0                                          
 frase    | Hola, quiero aprender algo de Spark        
 palabras | [hola,, quiero, aprender, algo, de, spark] 
-RECORD 1----------------------------------------------
 id       | 1                                          
 frase    | Me gustaría aprender algo de Java          
 palabras | [me, gustaría, aprender, algo, de, java]   
-RECORD 2----------------------------------------------
 id       | 2                                          
 frase    | suma,Resta,multiplicación                  
 palabras | [suma,resta,multiplicación]                



In [80]:
datosT.collect()

[Row(id=0, frase='Hola, quiero aprender algo de Spark', palabras=['hola,', 'quiero', 'aprender', 'algo', 'de', 'spark']),
 Row(id=1, frase='Me gustaría aprender algo de Java', palabras=['me', 'gustaría', 'aprender', 'algo', 'de', 'java']),
 Row(id=2, frase='suma,Resta,multiplicación', palabras=['suma,resta,multiplicación'])]

In [81]:
reTokenizer=RegexTokenizer(inputCol="frase",outputCol="palabras",pattern="[\s\,\-\_\.\;\(\)]+")

In [82]:
datosR=reTokenizer.transform(texto)

In [83]:
datosR.show()

+---+--------------------+--------------------+
| id|               frase|            palabras|
+---+--------------------+--------------------+
|  0|Hola, quiero apre...|[hola, quiero, ap...|
|  1|Me gustaría apren...|[me, gustaría, ap...|
|  2|suma,Resta,multip...|[suma, resta, mul...|
+---+--------------------+--------------------+



In [84]:
from pyspark.ml.feature import StopWordsRemover

In [85]:
swRemover=StopWordsRemover(inputCol="palabras",outputCol="filtradas")

In [86]:
palabrasBloqueadas=swRemover.loadDefaultStopWords("spanish")+["coche","resta"]

In [87]:
swRemover.setStopWords(palabrasBloqueadas)

StopWordsRemover_44a78280984426f096f5

In [88]:
filtrado=swRemover.transform(datosR)
filtrado.show(vertical=True,truncate=False)

-RECORD 0----------------------------------------------
 id        | 0                                         
 frase     | Hola, quiero aprender algo de Spark       
 palabras  | [hola, quiero, aprender, algo, de, spark] 
 filtradas | [hola, quiero, aprender, spark]           
-RECORD 1----------------------------------------------
 id        | 1                                         
 frase     | Me gustaría aprender algo de Java         
 palabras  | [me, gustaría, aprender, algo, de, java]  
 filtradas | [gustaría, aprender, java]                
-RECORD 2----------------------------------------------
 id        | 2                                         
 frase     | suma,Resta,multiplicación                 
 palabras  | [suma, resta, multiplicación]             
 filtradas | [suma, multiplicación]                    



In [89]:
from pyspark.ml.feature import NGram

In [90]:
ngram=NGram(n=2,inputCol="filtradas",outputCol="ngram")

In [91]:
datosN=ngram.transform(filtrado)

In [92]:
datosN.show(vertical=True,truncate=False)

-RECORD 0---------------------------------------------------
 id        | 0                                              
 frase     | Hola, quiero aprender algo de Spark            
 palabras  | [hola, quiero, aprender, algo, de, spark]      
 filtradas | [hola, quiero, aprender, spark]                
 ngram     | [hola quiero, quiero aprender, aprender spark] 
-RECORD 1---------------------------------------------------
 id        | 1                                              
 frase     | Me gustaría aprender algo de Java              
 palabras  | [me, gustaría, aprender, algo, de, java]       
 filtradas | [gustaría, aprender, java]                     
 ngram     | [gustaría aprender, aprender java]             
-RECORD 2---------------------------------------------------
 id        | 2                                              
 frase     | suma,Resta,multiplicación                      
 palabras  | [suma, resta, multiplicación]                  
 filtradas | [suma, mult

In [93]:
from pyspark.ml.feature import HashingTF

In [106]:
hTF=HashingTF(numFeatures=5000,inputCol="filtradas",outputCol="features")

In [107]:
datosH=hTF.transform(filtrado)

In [108]:
datosH.show(vertical=True,truncate=False)

-RECORD 0---------------------------------------------------
 id        | 0                                              
 frase     | Hola, quiero aprender algo de Spark            
 palabras  | [hola, quiero, aprender, algo, de, spark]      
 filtradas | [hola, quiero, aprender, spark]                
 features  | (5000,[2139,2596,2850,3105],[1.0,1.0,1.0,1.0]) 
-RECORD 1---------------------------------------------------
 id        | 1                                              
 frase     | Me gustaría aprender algo de Java              
 palabras  | [me, gustaría, aprender, algo, de, java]       
 filtradas | [gustaría, aprender, java]                     
 features  | (5000,[1407,2850,3967],[1.0,1.0,1.0])          
-RECORD 2---------------------------------------------------
 id        | 2                                              
 frase     | suma,Resta,multiplicación                      
 palabras  | [suma, resta, multiplicación]                  
 filtradas | [suma, mult